In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/star_classification.csv", sep=',')
output = data['class']
input = data.drop(['class','rerun_ID'], axis=1)

input_max = input.max()
input_min = input.min()
input_normalized = (input - input_min) / (input_max - input_min)
df = pd.DataFrame(input_normalized)

# # Compute the correlation matrix
correlation_matrix = df.corr()

# # Visualize the correlation matrix using a heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()
dataset = pd.concat([input_normalized, output], axis=1)

# Shuffle the dataset (by shuffling rows)
shuffled_dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Calculate the index to split the dataset (90% for training and 10% for testing)
train_size = int(0.9 * len(shuffled_dataset))

# Split the dataset into training and testing sets
train_data = shuffled_dataset[:train_size]
test_data = shuffled_dataset[train_size:]

# Separate features and labels for training and testing
input_normalized = train_data.drop('class', axis=1)
output = train_data['class']

X_test = test_data.drop('class', axis=1)
Y_test = test_data['class']

import matplotlib.pyplot as plt

# Plot histogram of the classes
output.value_counts().plot(kind='bar', color='skyblue')
plt.title('Class Distribution Before Oversampling Minority Cases')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.show()

# Check class distribution
class_counts = output.value_counts()
print(class_counts)

max_class_size = class_counts.max()
classes = class_counts.index

print(f"Majority Class Size: {max_class_size}")

# oversample
oversampled_data = []
# Loop through each class
for cls in classes:
    class_data = input_normalized[output == cls]  # Get data for the current class
    oversampled_class = class_data.sample(max_class_size, replace=True, random_state=42)  # Oversample with replacement
    oversampled_data.append(oversampled_class)

# Combine all oversampled classes
oversampled_input = pd.concat(oversampled_data, axis=0)

# Create the corresponding labels
oversampled_output = pd.Series(
    [cls for cls in classes for _ in range(max_class_size)],
    name='class'
)


#shuffle
# Reset indices of oversampled data and labels
oversampled_input = oversampled_input.reset_index(drop=True)
oversampled_output = oversampled_output.reset_index(drop=True)

# Combine input and output
balanced_data = pd.concat([oversampled_input, oversampled_output], axis=1)

# Shuffle the dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
input = data.drop(['class'], axis=1)

data = balanced_data

output = data['class']

input = data.drop(['class'], axis=1)


input_max = input.max()

input_min = input.min()

input_normalized = (input - input_min) / (input_max - input_min)

import matplotlib.pyplot as plt


balanced_data['class'].value_counts().plot(kind='bar', color='lightblue', edgecolor='black')
plt.title('Class Distribution After Oversampling')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/star_classification.csv'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import random

def one_hot_representation(class_array):
    data = sorted(list(set(class_array)))

    num_classes = len(data)
    one_hot = np.eye(num_classes)
    label_to_index = {label: idx for idx, label in enumerate(data)}

    one_hot_encoded = one_hot[[label_to_index[label] for label in class_array]]


    label = np.split(one_hot,one_hot.shape[1],axis=0)



    return one_hot_encoded , label

def loss(y_true, y_pred):

    epsilon = 1e-9  # Small value to prevent log(0)

    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)


    loss = -np.sum(y_true * np.log(y_pred)) / y_true.shape[0]
    return loss
def softmax(z):

    z_exp = np.exp(z - np.max(z, axis=1, keepdims=True))
    return z_exp / np.sum(z_exp, axis=1, keepdims=True)

def accuracy_score(y_test, y_pred):
    if len(y_test) != len(y_pred):
        raise ValueError("The length of y_test and y_pred must be the same.")
    y_test_class = np.argmax(y_test, axis=1)
    y_pred_class = np.argmax(y_pred, axis=1)
    correct_predictions = np.sum(y_test_class == y_pred_class)

    accuracy = correct_predictions / len(y_test)

    return accuracy
def compute_confusion_matrix(y_test , y_pred):

    Y_pred_classes = np.argmax(y_pred, axis=1)
    Y_test_classes = np.argmax(y_test, axis=1)
    conf_matrix = confusion_matrix(Y_test_classes, Y_pred_classes)

    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Galaxy","Star", "QSO"])
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix for Logistic Regression")
    plt.show()


    return conf_matrix

def f1(y_test, y_pred, class_labels):
    f1_scores = []

    for label in class_labels:
        TP = np.sum((y_test == label) & (y_pred == label))
        FP = np.sum((y_test != label) & (y_pred == label))
        FN = np.sum((y_test == label) & (y_pred != label))

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0

        f1_class = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1_class)

    return np.mean(f1_scores)

class LogisticRegression:
    def _init_(self, learning_rate=0.001, epoch=1000, num_classes=3, batch_size = 1):
        self.lr = learning_rate
        self.epoch = epoch
        self.classes = num_classes
        self.parameters = {}
        self.losses = []
        self.batch_size = batch_size


    def fit(self, X, y, batch_size=32, SGD=False ):

        n_samples, n_features = X.shape

        self.parameters["W1"] = np.random.randn(n_features, self.classes) * 0.01
        self.parameters["b1"] = np.zeros((1, self.classes))

        for epoch in range(self.epoch):
            if SGD:
                indices = np.random.permutation(n_samples)
                X_shuffled = X[indices]
                y_shuffled = y[indices]
                n_batch= X_shuffled.shape[0] // batch_size
                i = random.randint(0, n_batch )
                if i == n_batch:
                    xi = X_shuffled[(n_batch*batch_size)::]
                    yi = y_shuffled[(n_batch*batch_size)::]
                    dw, db = self.gradient(xi,yi)
                    self.update_parameters(dw,db)
                else:
                    xi = X_shuffled[(i*batch_size):((i+1)*batch_size)]
                    yi = y_shuffled[(i*batch_size):((i+1)*batch_size)]
                    dw, db = self.gradient(xi,yi)
                    self.update_parameters(dw,db)

            else:
                X_shuffled = X
                y_shuffled = y

                n_batch= X_shuffled.shape[0] // batch_size

                for i in range(n_batch):
                    xi = X_shuffled[(i*batch_size):((i+1)*batch_size)]
                    yi = y_shuffled[(i*batch_size):((i+1)*batch_size)]
                    dw, db = self.gradient(xi,yi)
                    self.update_parameters(dw,db)
                if X.shape[0] % batch_size != 0:
                    xi = X_shuffled[(n_batch*batch_size)::]
                    yi = y_shuffled[(n_batch*batch_size)::]

                    dw, db = self.gradient(xi,yi)
                    self.update_parameters(dw,db)

    def forward_pass(self,X):

        z = np.dot(X, self.parameters["W1"]) + self.parameters["b1"]

        A = softmax(z)
        return A



    def gradient(self, X, Y):
        A = self.forward_pass(X)
        self.losses.append(loss(Y, A))
        dz = A - Y

        dw = (1 / X.shape[0]) * np.dot(X.T, dz)
        db = (1 / X.shape[0]) * np.sum(dz)
        return dw, db
    def update_parameters(self,dw,db):
        self.parameters["W1"] = self.parameters["W1"] - self.lr * dw
        self.parameters["b1"] = self.parameters["b1"] - self.lr * db

    def predict(self, X):

        y_prediction = np.zeros((X.shape[0],self.classes))

        y_hat = np.dot(X, self.parameters["W1"]) + self.parameters["b1"]
        y_pred = softmax(y_hat)

        max_i = np.argmax(y_pred, axis=1)


        y_prediction[np.arange(len(y_pred)),max_i] = 1

        return np.array(y_prediction)

def k_fold(X, Y, k=5, shuffle=False,learning = (0.01, 0.001), epoch = (50, 100, 150) , batch_size=80000, sgd = (False, True)):
    if shuffle:
        indices = np.random.permutation(len(X))

        X_shuffled = X.iloc[indices]
        Y_shuffled = Y.iloc[indices]


    else:
        X_shuffled = X
        Y_shuffled = Y
    Y_shuffled_ohr, oh_labels = one_hot_representation(Y_shuffled)

    fold_size = len(X) // k

    cm = []
    model_accuracies = []
    model_f1_scores = []
    models = []
    #model_cm = []
    for m in range(len(epoch)):
        epochs = epoch[m]

        for j in range(len(learning)):
            learning_rate = learning[j]

            for l in range(len(sgd)):
                sgd_bool = sgd[l]
                accuracies = []
                f1_scores = []
                for i in range(k):

                    test_start = i * fold_size

                    test_end = (i + 1) * fold_size if i != k - 1 else len(X)

                    X_train = np.concatenate([X_shuffled[:test_start], X_shuffled[test_end:]], axis=0)
                    Y_train = np.concatenate([Y_shuffled_ohr[:test_start], Y_shuffled_ohr[test_end:]], axis=0)
                    X_valid = X_shuffled[test_start:test_end]
                    Y_valid = Y_shuffled_ohr[test_start:test_end]



                    model = LogisticRegression(learning_rate = learning_rate, epoch = epochs,num_classes = 3)

                    model.fit(X_train, Y_train, batch_size, SGD= sgd_bool)

                    Y_pred = model.predict(X_valid)

                    accuracy = accuracy_score(Y_valid, Y_pred)
                    accuracies.append(accuracy)
                    f1_score = f1(Y_valid, Y_pred, oh_labels)
                    f1_scores.append(f1_score)
                average_accuracy = np.mean(accuracies)
                average_f1_score = np.mean(f1_scores)
                cm.append(compute_confusion_matrix(Y_valid, Y_pred))
                model_accuracies.append(average_accuracy)
                model_f1_scores.append(average_f1_score)
                models.append(model)
                print("Logistic Regression: ")
                print("For learning rate " + str(learning_rate) + " and epoch " + str(epochs) + " and sgd " + str(sgd_bool))
                print(f"Validation Accuracy: {average_accuracy}")
                print(f"Validation F1 Score: {average_f1_score}\n")


    #model_cm.append(cm)
    return  model_accuracies, model_f1_scores, models




In [ ]:
Y_test_ohr, oh_lab = one_hot_representation(Y_test)

Acc, F1, models = k_fold(input_normalized, output, shuffle=True, k=5, learning=(0.01,) ,epoch=(100,), batch_size=1, sgd = ( False,))
print("Validation Accuracies are " + str(Acc))
print("Validation F1 Scores are " + str(F1))

best = np.argmax(F1)
best_model = models[best]

predictions = best_model.predict(X_test)
accuracy = accuracy_score(Y_test_ohr, predictions)
f1_score = f1(Y_test_ohr, predictions, oh_lab)
cm = (compute_confusion_matrix(Y_test_ohr, predictions))
print("Test Accuracy: " + str(accuracy))
print("Test F1 Score: " + str(f1_score))